# Simulation example using Area Yield DGP

In [1]:
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm

from sklearn.ensemble import StackingRegressor, StackingClassifier
from sklearn.linear_model import RidgeCV
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression

from rdrobust import rdrobust, rdbwselect

import doubleml as dml
from doubleml.utils import GlobalRegressor, GlobalClassifier
from doubleml.rdd import RDFlex
from doubleml.rdd import datasets

from matplotlib import pyplot as plt
from statsmodels.nonparametric.kernel_regression import KernelReg

import plotly.express as px

In [2]:
%run utils.py

## DGP Parameters

In [3]:
cutoff_dist = 0.45
cutoff_improvement = 0.0

params = dict(
    seed=17,
    n_obs=3,
    K=100,
    # origin
    origin_shape='ellipsis',
    origin_a=0.035,
    origin_b=0.01,
    origin_pertubation=0.2,
    # target
    target_center=[1.5, 0],
    target_a=0.6,
    target_b=0.3,
    # action
    action_shift=[1.0, 0],
    action_scale=1.02,
    #action_pertubation=None,
    action_pertubation=[0.001, 0.0006],
    action_drag_share=0.7,
    action_drag_scale=0.7,
    # running
    running_dist_measure='projected',
    running_mea_selection=5,
    # treatment
    treatment_dist=cutoff_dist,
    treatment_improvement=cutoff_improvement,
    treatment_random_share=0.0,
)

## DGP Visualization

In [4]:
data = datasets.dgp_area_yield(**{**params, 'n_obs': 5000, 'seed': 8})

# improved_points = data['score_improvement'] >= 0.0
# data = {k:v[improved_points] for k,v in data.items()}

fig = px.scatter(x=data['score_distance'] ,y=data['Y'], color=data['D'], labels=dict(x="score_distance", y="Y"), hover_data={'score_improvement': data['score_improvement'], 'score_distance_act': data['score_distance_act'], 'score_improvement_act': data['score_improvement_act']}) 
fig.add_vline(x=params['treatment_dist'])

In [5]:
fig = px.scatter(x=data['score_distance'] ,y=data['score_improvement'], color=data['D'], labels=dict(x="score_distance", y="score_improvement"), hover_data={'score_improvement': data['score_improvement'], 'score_distance_act': data['score_distance_act'], 'score_improvement_act': data['score_improvement_act']}) 
fig.add_vline(x=params['treatment_dist'])
fig.add_hline(y=cutoff_improvement)

In [6]:
fig = px.scatter(x=data['score_improvement'] ,y=data['Y1'] - data['Y0'], color=data['D'], labels=dict(x="score_improvement", y="effect"), hover_data={'score_improvement': data['score_improvement'], 'score_distance_act': data['score_distance_act'], 'score_improvement_act': data['score_improvement_act']}) 
fig.add_vline(x=params['treatment_improvement'])

In [7]:
fig = px.scatter(x=data['score_distance'] ,y=data['Y1'] - data['Y0'], color=data['D'], labels=dict(x="score_distance", y="effect"), hover_data={'score_improvement': data['score_improvement'], 'score_distance_act': data['score_distance_act'], 'score_improvement_act': data['score_improvement_act']}) 
fig.add_vline(x=params['treatment_dist'])

## Oracle / Neighborhood Estimator
Estimation is done using bigger sample

In [8]:
n_obs = 10000
selected_params = {**params, 'n_obs': n_obs}
data = datasets.dgp_area_yield(**selected_params)

defiers = data["D"] != data["T"]
print(f"Defier percentage: {defiers.mean()}")

Defier percentage: 0.0211


In [9]:
cutoff = params['treatment_dist']
score = data['score_distance']
ite = data['Y1'] - data['Y0']

kernel_regression_range = 0.3
# remove defiers and restrict to a range around the cutoff
kernel_subset = (score >= cutoff - kernel_regression_range) & (score <= cutoff + kernel_regression_range)

Define all oracle effects

In [10]:
# fuzzy effect
kernel_subset_fuzzy = kernel_subset & ~defiers
kernel_reg_fuzzy = KernelReg(endog=ite[kernel_subset_fuzzy], exog=score[kernel_subset_fuzzy], var_type='c', reg_type='ll')

effect_fuzzy, _  = kernel_reg_fuzzy.fit(np.array([cutoff]))
effect_fuzzy_kernel = effect_fuzzy[0]
print(f"Estimated effect at cutoff (fuzzy): {effect_fuzzy_kernel}")

# intention to treat
ite_intend = data['Y1'] - data['Y0']
ite_intend[defiers] = 0 # only if defiers all right to the cutoff (else negative effect)
kernel_reg_intend = KernelReg(endog=ite_intend[kernel_subset], exog=score[kernel_subset], var_type='c', reg_type='ll')

effect_intend, _  = kernel_reg_intend.fit(np.array([cutoff]))
effect_intend_kernel = effect_intend[0]
print(f"Estimated effect at cutoff (intend to treat): {effect_intend_kernel}")

Estimated effect at cutoff (fuzzy): 0.044818358947511655
Estimated effect at cutoff (intend to treat): 0.033941916030289225


In [11]:
treatment_dist = params['treatment_dist']
X1_close_fuzzy = (score[kernel_subset_fuzzy] > treatment_dist - 0.02) & (score[kernel_subset_fuzzy] < treatment_dist + 0.02)
print(f'Neighborhood observations (fuzzy): {X1_close_fuzzy.sum()}')
effect_fuzzy_neighborhood = ite[kernel_subset_fuzzy][X1_close_fuzzy].mean()
print(f'Neighborhood estimator (fuzzy): {effect_fuzzy_neighborhood}')

X1_close = (score[kernel_subset] > treatment_dist - 0.02) & (score[kernel_subset] < treatment_dist + 0.02)
print(f'Neighborhood observations (intend to treat): {X1_close.sum()}')
effect_intend_neighborhood = ite_intend[kernel_subset][X1_close].mean()
print(f'Neighborhood estimator (intend to treat): {effect_intend_neighborhood}')

Neighborhood observations (fuzzy): 165
Neighborhood estimator (fuzzy): 0.0470909090909091
Neighborhood observations (intend to treat): 209
Neighborhood estimator (intend to treat): 0.037177033492822975


## Learners

In [12]:
base_regressors = [
    ('lgbm_regressor', LGBMRegressor(n_estimators=200, learning_rate=0.01, verbose=-1, n_jobs=-1)),
    ('linear_regressor', LinearRegression()),
    ('global_regressor', GlobalRegressor(LinearRegression())),
]

stacking_regressor = StackingRegressor(
    estimators=base_regressors,
    final_estimator=LinearRegression(),
    n_jobs=-1
)

base_classifiers = [
    ('lgbm_classifier', LGBMClassifier(n_estimators=200, learning_rate=0.01, verbose=-1, n_jobs=-1)),
    ('logistic_classifier', LogisticRegression()),
    ('global_classifier', GlobalClassifier(LogisticRegression())),
]

stacking_classifier = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=LogisticRegression(),
    n_jobs=-1
)

In [13]:
linear_learner_dict = {
    "regressor": LinearRegression(),
    "classifier": LogisticRegression(),
}

global_linear_learner_dict = {
    "regressor": GlobalRegressor(LinearRegression()),
    "classifier": GlobalClassifier(LogisticRegression()),
}

lgbm_learner_dict = {
    "regressor": LGBMRegressor(n_estimators=200, learning_rate=0.01, verbose=-1, n_jobs=-1),
    "classifier": LGBMClassifier(n_estimators=200, learning_rate=0.01, verbose=-1, n_jobs=-1),
}


In [14]:
learner_dict = {
    "linear": linear_learner_dict,
    "global_linear": global_linear_learner_dict,
    "lgbm": lgbm_learner_dict,
}

## Simulation

### Single replication

In [15]:
from scipy.stats import skew, kurtosis

In [16]:
def generate_data(seed, n_obs=1000):
    # generate data
    selected_params = {**params, 'n_obs': n_obs, 'seed': seed}
    data = datasets.dgp_area_yield(**selected_params)
    score = data["score_distance"]
    Y = data["Y"]
    D = data["D"]

    # Replace X with non score-related parameters
    # TODO: Maybe add range or offset from conversion curve
    mean_values = np.mean(data["X"], axis=1)
    min_values = np.min(data["X"], axis=1)
    max_values = np.max(data["X"], axis=1)
    std_values = np.std(data["X"], axis=1)
    skewness_values = skew(data["X"], axis=1)
    kurtosis_values = kurtosis(data["X"], axis=1)

    X = np.column_stack((mean_values, min_values, max_values, std_values, skewness_values, kurtosis_values))
    # X = data["X"][:, :-1, :].reshape(n_obs, -1)
    # X = data["X"].reshape(n_obs, -1)

    data_dict = {
        "X": X,
        "D": D,
        "Y": Y,
        "score": score,
    }
    return data_dict

def estimate_fuzzy(seed, data_dict, learner_dict, fs_specifications, cutoff=0):
    res_list_fuzzy = []
    
    # run basic rdrobust
    try:
        res_fuzzy = rdrobust(y=data_dict["Y"], x=data_dict["score"], fuzzy=data_dict["D"], covs=data_dict["X"], c=cutoff)
        h=None
    except Exception as e:
        print(f"Error in rdrobust bandwidth: {e}")
        h = rdbwselect(y=data_dict["Y"],
            x=data_dict["score"],
            fuzzy=data_dict["D"],
            c=cutoff,
            sharpbw=True).bws.values.flatten().max()
        res_fuzzy = rdrobust(y=data_dict["Y"], x=data_dict["score"], fuzzy=data_dict["D"], covs=data_dict["X"], c=cutoff, h=h)
    res_list_fuzzy.append(
        {"rep": seed, "method": "rdrobust", "learner": "linear",
         "orcl": effect_fuzzy_kernel, "orcl_neigh": effect_fuzzy_neighborhood,
         "fs_specification": "cutoff",
         "coef": res_fuzzy.coef.loc["Conventional", "Coeff"], "se": res_fuzzy.se.loc["Robust", "Std. Err."], 
         "2.5 %": res_fuzzy.ci.loc["Robust", "CI Lower"], "97.5 %": res_fuzzy.ci.loc["Robust", "CI Upper"]})

    # RDFlex methods
    dml_data = dml.DoubleMLData.from_arrays(y=data_dict["Y"], d=data_dict["D"], x=data_dict["X"], s=data_dict["score"])

    for learner_name, learners in learner_dict.items():
        for fs_specification in fs_specifications:
            rdflex_model_fuzzy = RDFlex(dml_data,
                                        ml_g=learners["regressor"],
                                        ml_m=learners["classifier"],
                                        n_folds=5,
                                        n_rep=1,
                                        cutoff=cutoff,
                                        fuzzy=True,
                                        fs_specification=fs_specification,
                                        h_fs=h)
            rdflex_model_fuzzy.fit(n_iterations=2)
            res_list_fuzzy.append(
                {"rep": seed, "method": "rdflex", "learner": learner_name,
                "orcl": effect_fuzzy_kernel, "orcl_neigh": effect_fuzzy_neighborhood,
                "fs_specification": fs_specification,
                "coef": rdflex_model_fuzzy.coef[0], "se": rdflex_model_fuzzy.se[2], 
                "2.5 %": rdflex_model_fuzzy.confint().loc["Robust", "2.5 %"], "97.5 %": rdflex_model_fuzzy.confint().loc["Robust", "97.5 %"]})
    return res_list_fuzzy


def estimate_indend(seed, data_dict, learner_dict, fs_specifications, cutoff=0):
    res_list_intend = []

    res_intend = rdrobust(y=data_dict["Y"], x=data_dict["score"], covs=data_dict["X"], c=cutoff)
    res_list_intend.append(
        {"rep": seed, "method": "rdrobust", "learner": "linear",
         "orcl": effect_intend_kernel, "orcl_neigh": effect_intend_neighborhood,
         "fs_specification": "interacted cutoff and score",
         "coef": res_intend.coef.loc["Conventional", "Coeff"], "se": res_intend.se.loc["Robust", "Std. Err."], 
         "2.5 %": res_intend.ci.loc["Robust", "CI Lower"], "97.5 %": res_intend.ci.loc["Robust", "CI Upper"]})
    
    # RDFlex methods
    dml_data = dml.DoubleMLData.from_arrays(y=data_dict["Y"], d=data_dict["D"], x=data_dict["X"], s=data_dict["score"])

    for learner_name, learners in learner_dict.items():
        for fs_specification in fs_specifications:

            rdflex_model_intend = RDFlex(dml_data,
                                        ml_g=learners["regressor"],
                                        n_folds=5,
                                        n_rep=1,
                                        cutoff=cutoff,
                                        fuzzy=False,
                                        fs_specification=fs_specification)
            rdflex_model_intend.fit(n_iterations=2)
            res_list_intend.append(
                {"rep": seed, "method": "rdflex", "learner": learner_name,
                "orcl": effect_intend_kernel, "orcl_neigh": effect_intend_neighborhood,
                "fs_specification": fs_specification,
                "coef": rdflex_model_intend.coef[0], "se": rdflex_model_intend.se[2],
                "2.5 %": rdflex_model_intend.confint().loc["Robust", "2.5 %"], "97.5 %": rdflex_model_intend.confint().loc["Robust", "97.5 %"]})
    
    return res_list_intend

In [17]:
n_rep = 1
fs_specifications = ["cutoff"]

res_list_fuzzy = []
res_list_intend = []
for r in tqdm(range(n_rep), desc="Repetitions", unit="rep"):

    data_dict = generate_data(r, n_obs=5000)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        # Fuzzy RDD
        try:
            single_res_fuzzy = estimate_fuzzy(
                r,
                data_dict,
                learner_dict=learner_dict,
                fs_specifications=fs_specifications,
                cutoff=cutoff)
            res_list_fuzzy.extend(single_res_fuzzy)
        except Exception as e:
                print(f"An error occurred during fuzzy repetition {r}: {e}")

        # Intention to treat RDD
        try:
            single_res_intend = estimate_indend(
                r,
                data_dict,
                learner_dict=learner_dict,
                fs_specifications=fs_specifications,
                cutoff=cutoff)
            res_list_intend.extend(single_res_intend)
        except Exception as e:
                print(f"An error occurred during intend repetition {r}: {e}")

df_fuzzy = pd.DataFrame(res_list_fuzzy)
df_intend = pd.DataFrame(res_list_intend)

Repetitions: 100%|██████████| 1/1 [00:18<00:00, 18.48s/rep]


In [18]:
df_fuzzy.head(n=10)

,rep,method,learner,orcl,orcl_neigh,fs_specification,coef,se,2.5 %,97.5 %
0,0,rdrobust,linear,0.044818,0.047091,cutoff,0.099111,0.022317,0.075472,0.162952
1,0,rdflex,linear,0.044818,0.047091,cutoff,0.145555,0.048045,0.093835,0.282168
2,0,rdflex,global_linear,0.044818,0.047091,cutoff,0.130791,0.044233,0.094949,0.268339
3,0,rdflex,lgbm,0.044818,0.047091,cutoff,0.121635,0.054061,0.129237,0.341152


In [19]:
df_intend.head(n=10)

,rep,method,learner,orcl,orcl_neigh,fs_specification,coef,se,2.5 %,97.5 %
0,0,rdrobust,linear,0.033942,0.037177,interacted cutoff and score,0.049816,0.012365,0.030646,0.079117
1,0,rdflex,linear,0.033942,0.037177,cutoff,0.050200,0.016217,0.040923,0.104493
2,0,rdflex,global_linear,0.033942,0.037177,cutoff,0.047820,0.016467,0.039862,0.104411
3,0,rdflex,lgbm,0.033942,0.037177,cutoff,0.056221,0.021910,0.041550,0.127436


In [20]:
n_rep = 100
fs_specifications = ["cutoff"]

res_list_fuzzy = []
res_list_intend = []
for r in tqdm(range(n_rep), desc="Repetitions", unit="rep"):

    data_dict = generate_data(r, n_obs=5000)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        # Fuzzy RDD
        try:
            single_res_fuzzy = estimate_fuzzy(
                r,
                data_dict,
                learner_dict=learner_dict,
                fs_specifications=fs_specifications,
                cutoff=cutoff)
            res_list_fuzzy.extend(single_res_fuzzy)
        except Exception as e:
                print(f"An error occurred during fuzzy repetition {r}: {e}")

        # Intention to treat RDD
        try:
            single_res_intend = estimate_indend(
                r,
                data_dict,
                learner_dict=learner_dict,
                fs_specifications=fs_specifications,
                cutoff=cutoff)
            res_list_intend.extend(single_res_intend)
        except Exception as e:
                print(f"An error occurred during intend repetition {r}: {e}")

df_fuzzy = pd.DataFrame(res_list_fuzzy)
df_intend = pd.DataFrame(res_list_intend)

Repetitions: 100%|██████████| 100/100 [29:01<00:00, 17.42s/rep]


## Evaluation

In [21]:
def evaluate_results(df):
    df["CI width"] = df["97.5 %"] - df["2.5 %"]
    df["Coverage"] = (df["2.5 %"] <= df["orcl"]) & (df["97.5 %"] >= df["orcl"])
    df["Coverage_neigh"] = (df["2.5 %"] <= df["orcl_neigh"]) & (df["97.5 %"] >= df["orcl_neigh"])

    df["centered coef"] = df["coef"] - df["orcl"]

    print(80*"=")
    print(f"Coverage:\n {df.groupby(["method", "learner", "fs_specification"])["Coverage"].mean()}")
    print(80*"=")
    print(f"Mean CI width:\n {df.groupby(["method", "learner", "fs_specification"])["CI width"].mean()}")
    print(f"Median CI width:\n {df.groupby(["method", "learner", "fs_specification"])["CI width"].median()}")
    print(80*"=")

    return df


In [22]:
_ = evaluate_results(df_fuzzy)

Coverage:
 method    learner        fs_specification
rdflex    global_linear  cutoff              0.24
          lgbm           cutoff              0.18
          linear         cutoff              0.23
rdrobust  linear         cutoff              0.06
Name: Coverage, dtype: float64
Mean CI width:
 method    learner        fs_specification
rdflex    global_linear  cutoff              0.221401
          lgbm           cutoff              0.225076
          linear         cutoff              0.218627
rdrobust  linear         cutoff              0.073913
Name: CI width, dtype: float64
Median CI width:
 method    learner        fs_specification
rdflex    global_linear  cutoff              0.212353
          lgbm           cutoff              0.171935
          linear         cutoff              0.205956
rdrobust  linear         cutoff              0.073811
Name: CI width, dtype: float64


In [23]:
_ = evaluate_results(df_intend)

Coverage:
 method    learner        fs_specification           
rdflex    global_linear  cutoff                         0.70
          lgbm           cutoff                         0.78
          linear         cutoff                         0.64
rdrobust  linear         interacted cutoff and score    0.59
Name: Coverage, dtype: float64
Mean CI width:
 method    learner        fs_specification           
rdflex    global_linear  cutoff                         0.053589
          lgbm           cutoff                         0.069425
          linear         cutoff                         0.053416
rdrobust  linear         interacted cutoff and score    0.042968
Name: CI width, dtype: float64
Median CI width:
 method    learner        fs_specification           
rdflex    global_linear  cutoff                         0.053408
          lgbm           cutoff                         0.069441
          linear         cutoff                         0.052946
rdrobust  linear         interacted